# Task
We want to achieve the following workflow:

Step 1. 
1. Write "Hello" in a file 

**However, this is too simple for dflow. We want to write "Hello" in 10 different files.**

In [4]:
from typing import List

from dflow import Step, Workflow, argo_range
from dflow.python import OP, OPIO, Artifact, OPIOSign, PythonOPTemplate, Slices

In [5]:
class Hello(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            'filename': str
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign({
            'out_art': Artifact(str)
        })

    @OP.exec_sign_check
    def execute(
            self,
            op_in: OPIO,
    ) -> OPIO:
        open(op_in["filename"], "w").write("Hello")
        op_out = OPIO({
            'out_art': op_in["filename"]
        })
        return op_out

In [6]:
wf = Workflow("slices")
hello = Step("hello",
            PythonOPTemplate(Hello, image="python:3.8",
                            slices=Slices("{{item}}",
                                        input_parameter=["filename"],
                                        output_artifact=["out_art"]
                                        )
                            ),
            parameters={"filename": [f"f{x}.txt" for x in range(10)]},
            with_param=argo_range(10))
wf.add(hello)
wf.submit();


Workflow has been submitted (ID: slices-d9rmk)
